In [44]:
import fitz
import sys, os, re, unicodedata
from intervaltree import Interval, IntervalTree
from pprint import pprint

In [45]:
def flags_decomposer(flags: int) -> str:
    # Make font flags human readable
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)

def parse(pdf: fitz.Document) -> (str, IntervalTree):
    s, idx, it = '', 0, IntervalTree()
    for page in pdf:
        tdict = page.get_text(
            'dict',
            flags=  fitz.TEXTFLAGS_DICT
                 & ~fitz.TEXT_PRESERVE_IMAGES
        )
        for block in tdict['blocks']:
            for line_no, line in enumerate(block['lines']):
                line_start_idx = idx
                for span in line['spans']:
                    t = span['text']
                    it[idx:idx+len(t)] = (page.number, block['number'], line_no, idx, line_start_idx, span)
                    idx += len(t)
                    s += t
                idx += 1
                s += '\n'
    return s, it


In [46]:
pdf = fitz.open('ieee.pdf')

In [47]:
s, it = parse(pdf)
f = open('a.txt', 'w')
words_on_page = []
for refs_pat in pdf.pages():
    d = {}
    ws = refs_pat.get_text('words')
    for t in ws:
        x0, y0, x1, y1, w, block_no, line_no, word_no = t
        d[(block_no, line_no, word_no)] = (x0, y0, x1, y1, w)
        try:
            f.write(str((block_no, line_no, word_no)) + ': ' + str(w) + '\n')
        except:
            pass
    words_on_page.append(d)
f.close()

In [48]:
ieee_ref = r'[1-9][0-9]*'
#ieee_refs = r'\[{x}(,\s*{x})*\]'.format(x=ieee_ref)
ieee_refs = r'\({x}\)'.format(x=ieee_ref)
ref_pat = re.compile(ieee_ref)
refs_pat = re.compile(ieee_refs)

for m in refs_pat.finditer(s):
    for n in ref_pat.finditer(m.group()):
        start, end = n.span()
        start, end = start + m.start(), end + m.start()
        res = list(it.overlap(start, end))
        if len(res) != 1:
            print(len(res))
        page, blk_no, line_no, base, line_base, span = res[0].data
        
        tdict = pdf[page].get_text(
            'dict',
            flags=  fitz.TEXTFLAGS_DICT
                 & ~fitz.TEXT_PRESERVE_IMAGES
        )
        line = ''
        for s in tdict['blocks'][blk_no]['lines'][line_no]['spans']:
            line += s['text']
        #t = span['text'].replace('\n', '')
        t = line
        print(t)

        relative_idx = start - line_base
        word_idx = ' '.join(t.split())[:relative_idx].count(' ')

        word_base = relative_idx
        while word_base > 0 and t[word_base - 1] != ' ':
            word_base -= 1
        word_base += line_base

        try:
            x0, y0, x1, y1, w = words_on_page[page][(blk_no, line_no, word_idx)]
        except:
            print((blk_no, line_no, word_idx))
            print(t)
            print(n.group())
        
        x = x1 - x0
        hx0 = x0 + (start - word_base) / len(w) * x
        hx1 = hx0 + (end - start) / len(w) * x
        pdf[page].add_highlight_annot([hx0, y0, hx1, y1])

pdf.save('out.pdf')

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/jupyter_client/session.py", line 100, in json_packer
    ).encode("utf8", errors="surrogateescape")
      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'utf-8' codec can't encode character '\udc50' in position 32: surrogates not allowed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/zmq/eventloop/zmqstream.py", line 584, in _run_callback
    f = callback(*args, **kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/ipykernel/iostream.py", line 159, in _handle_event
    event_f()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/iostream.py", line 604, in _flush
    self.session.send(
  File "/usr/local/lib/python3.11/site-packages/jupyter_client/session.py", line 850, in send
   

In [49]:
page = pdf[1]
tdict = page.get_text('dict', flags=fitz.TEXTFLAGS_DICT & ~fitz.TEXT_PRESERVE_IMAGES)
with open('out.py', 'w') as f:
    pprint(tdict, f)